In [41]:
from libs.search import *
import matplotlib.pyplot as plt
from PIL import Image
import random
import time
import math



In [42]:
def plot_NQueens(solution, fig=None):
    n = len(solution)
    board = np.array([2 * int((i + j) % 2) for j in range(n) for i in range(n)]).reshape((n, n))
    im = Image.open('images/queen_s.png')
    height = im.size[1]
    im = np.array(im).astype(float) / 255
    fig = plt.figure(figsize=(7, 7)) if fig is None else fig
    ax = fig.add_subplot(111)
    ax.set_title('{} Queens'.format(n))
    plt.imshow(board, cmap='binary', interpolation='nearest')
    if isinstance(solution, dict):
        for (k, v) in solution.items():
            newax = fig.add_axes([0.064 + (k * 0.112), 0.062 + ((n - v) * 0.112), 0.1, 0.1], zorder=1)
            newax.imshow(im)
            newax.axis('off')
    elif isinstance(solution, list):
        for (k, v) in enumerate(solution):
            newax = fig.add_axes([0.064 + (k * 0.896/n), ((n - v) * 0.896/n) - 0.5/n + n*0.002, 0.8/n, 0.8/n], zorder=1)
            newax.imshow(im)
            newax.axis('off')
    fig.tight_layout()
    plt.show()

In [43]:
def objective_function(board):
    """Compute the total number of queens attacking each other."""
    attacks = 0
    n = len(board)
    for i in range(n):
        for j in range(i + 1, n):
            if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                attacks += 1
    return attacks

def successor(board):
    """Generate a new board configuration by randomly moving queens to valid positions within their columns."""
    new_board = list(board)
    for i in range(len(new_board)):
        attempts = 0
        while attempts < 10:  # Limit the number of attempts to avoid infinite loops
            new_position = random.randint(0, len(new_board) - 1)
            if new_position != new_board[i] and not isattacking(new_board, i, new_position):  # Ensure new position is not occupied and not attacking other queens
                new_board[i] = new_position
                
                break
            attempts += 1
    return new_board

def isattacking(board, row, col):
    for i in range(row):
        if board[i] == col or abs(board[i] - col) == abs(i - row):
            return True
    return False


def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost < old_cost:
        return 1.0
    return math.exp((old_cost - new_cost) / temperature)




In [44]:
def hill_climbing(board):
    
    """Hill climbing algorithm."""
    current_board = list(board)
    current_value = objective_function(current_board)
    iterations = 0
    
    while True:
        iterations += 1 
        neighbors= [successor(current_board) for _ in range(len(current_board))]
      
        print(neighbors)
        next_board = min(neighbors, key=objective_function)
        next_value = objective_function(next_board)
        
        if next_value >= current_value:
            return current_board,iterations
        
        current_board = next_board
        current_value = next_value

def stochastic_hill_climbing(initial_state, max_iterations=1000):
   
    current_board = initial_state
    current_value = objective_function(current_board)
    iterations = 0
    counter=0
    while iterations < max_iterations:
        counter+=1
        neighbors = [successor(current_board) for _ in range(len(current_board))]
       
        valid_neighbors = [neighbor for neighbor in neighbors if objective_function(neighbor) < current_value]
        
        if not valid_neighbors:
            break
        
        next_board = random.choice(valid_neighbors)
        next_value = objective_function(next_board)
        
        if next_value >= current_value:
            break
        
        current_board = next_board
        current_value = next_value
        iterations += 1
    
    return current_board,counter

def simulated_annealing(initial_state, max_iterations=1000, initial_temperature=1.0, cooling_rate=0.95):
   
    current_state = initial_state
    iterations=0
    current_cost = objective_function(current_state)
    best_state = current_state
    best_cost = current_cost
    temperature = initial_temperature

    while current_cost > 0 and iterations < max_iterations:
        iterations+=1
        new_state = successor(current_state)
        
        new_cost = objective_function(new_state)

        if acceptance_probability(current_cost, new_cost, temperature) > random.random():
            current_state = new_state
            current_cost = new_cost

        if new_cost < best_cost:
            best_state = new_state
            best_cost = new_cost

        temperature *= cooling_rate

    return best_state, best_cost,iterations


In [45]:
seed_value = 1100
random.seed(seed_value)
initial_state = [random.randint(0, 7) for _ in range(8)]

plot_NQueens(initial_state)
hill_climbing_solution,time1= hill_climbing(initial_state)
plot_NQueens(hill_climbing_solution)
stochastic_hill_climbing_solution ,time2 = stochastic_hill_climbing(initial_state)
plot_NQueens(stochastic_hill_climbing_solution)
best_state, best_cost,time3 = simulated_annealing(initial_state)
plot_NQueens(best_state)

print('Intial state - ', initial_state)

print('Hill Climbing Solution - ', hill_climbing_solution)
print('Time Complexity :', time1)

print('Stochastic Hill ClimbingSolution - ', stochastic_hill_climbing_solution)
print('Time Complexity :', time2)

print('Simulayed Annealing Best state:', best_state)
print('Best cost:', best_cost)
print('Time Complexity :', time3)


[[6, 0, 7, 5, 6, 2, 3, 3], [1, 6, 0, 5, 7, 4, 3, 3], [2, 7, 5, 3, 1, 4, 3, 3], [1, 5, 7, 2, 0, 3, 6, 4], [4, 1, 7, 2, 6, 3, 0, 3], [1, 6, 0, 3, 7, 4, 3, 3], [6, 0, 3, 5, 7, 2, 3, 3], [4, 7, 3, 6, 2, 5, 1, 3]]
[[3, 6, 2, 5, 1, 3, 7, 4], [7, 1, 4, 6, 0, 3, 5, 4], [2, 6, 3, 0, 4, 1, 6, 4], [5, 3, 6, 4, 7, 1, 6, 4], [5, 1, 6, 4, 7, 3, 6, 2], [3, 6, 2, 5, 0, 3, 6, 4], [6, 1, 3, 5, 0, 2, 4, 4], [6, 1, 5, 0, 0, 7, 4, 2]]
Intial state -  [0, 3, 2, 0, 6, 2, 3, 3]
Hill Climbing Solution -  [1, 5, 7, 2, 0, 3, 6, 4]
Time Complexity : 2
Stochastic Hill ClimbingSolution -  [1, 7, 5, 0, 2, 4, 6, 3]
Time Complexity : 3
Simulayed Annealing Best state: [3, 6, 4, 2, 0, 5, 7, 1]
Best cost: 0
Time Complexity : 36
